In [1]:
%%capture
!pip install transformers
!pip install torch
!pip install python-Levenshtein

In [2]:
from transformers import  AutoTokenizer
from transformers import RobertaForTokenClassification, RobertaForMaskedLM, RobertaTokenizer
from transformers import pipeline
import numpy as np
import torch

In [3]:
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base', model_max_length=512)
model = RobertaForTokenClassification.from_pretrained("/content/drive/MyDrive/Colab Notebooks/Trained model", local_files_only=True)
model_masked = RobertaForMaskedLM.from_pretrained('xlm-roberta-base')

You are using a model of type xlm-roberta to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.


In [4]:
def top_n_alternative(inputs, n):
  token_logits = model_masked(**inputs).logits
  mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
  mask_token_logits = token_logits[0, mask_token_index, :]
  top_n_tokens = torch.topk(mask_token_logits, n, dim=1)
  return top_n_tokens

In [5]:
from Levenshtein import distance as levenshtein_distance

def procentage_similarity(Q, Mi):
  levDis = levenshtein_distance(Q, Mi)
  bigger = max(len(Q), len(Mi))
  return (bigger - levDis) / bigger

In [31]:
import copy

def correct(text):
  inputs = tokenizer(text, return_tensors="pt")
  with torch.no_grad():
    logits = model(**inputs).logits

  predicted_token_class_ids = logits.argmax(-1)
  predicted_token_class_ids[0]

  context_dic = {}
  current_token = 0

  for i in range(len(predicted_token_class_ids[0])):
    if predicted_token_class_ids[0][i] == 1:
      
      word_id = inputs.word_ids()[i]

      word_list = []

      for j in range(len(inputs.word_ids())):
        if inputs.word_ids()[j] == word_id:
          word_list.append(inputs['input_ids'][0][j])
      

      context_dic[current_token] = tokenizer.decode(word_list)
      current_token += 1

  i=0
  inputs_list = inputs['input_ids'][0].tolist()
  predicted_token_list = predicted_token_class_ids[0].tolist()

  while i < len(inputs_list):
    if predicted_token_list[i] == 2:
        del inputs_list[i]
        del predicted_token_list[i]
    else:
      i += 1

  inter = tokenizer.decode(inputs_list[1:-1])
  input_2 = tokenizer(inter)

  lists = []

  for i in range(len(predicted_token_list)):
    if predicted_token_list[i] == 1:
      aux = copy.deepcopy(input_2['input_ids'])
      aux[i] = 250001
      lists.append(aux)

  word_ids = input_2.word_ids()

  best_scores = []

  for k in range(len(lists)):

    word_id = None
    for i in range(len(word_ids)):
      if lists[k][i] == 250001:
        word_id = word_ids[i]


    tmp = lists[k][1:-1]
    decoded = tokenizer.decode(tmp)
    inpt = tokenizer(decoded, return_tensors="pt")
    top_100 = top_n_alternative(inpt, 100)
    top_100_score = torch.softmax(top_100[0], dim=1)
    top_100_tokens = top_100[1]
    best_score = {'score': 0, 'lst':[]}

    for i in range(len(top_100_tokens[0])):
      aux = copy.deepcopy(lists[k])
      score_model = top_100_score[0][i]
      
      for id in range(len(aux)):
        if aux[id] == 250001:
          aux[id] = top_100_tokens[0][i]

      
      context_list = []
      for i in range(len(word_ids)):
        if(word_id == word_ids[i]):
          context_list.append(aux[i])
          
      new_context = tokenizer.decode(context_list)
      score_similarity = procentage_similarity(new_context, context_dic[k])
      final_score = (score_similarity + score_model) / 2

      if final_score > best_score['score']:
        best_score['score'] = final_score
        best_score['lst'] = aux
    
    best_scores.append(best_score)


  final_phrase = []

  k = 0
  start = False


  if context_dic != {}:
    if(len(best_scores) == 1):
      return tokenizer.decode(best_scores[0]['lst'][1:-1])
    else:
      for i in range(len(predicted_token_list)):
        if predicted_token_list[i] == 1:
          final_phrase.append(best_scores[k]['lst'][i])
          k += 1
        else:
          final_phrase.append(best_scores[0]['lst'][i])
      return tokenizer.decode(final_phrase[1:-1])
  else:
    return tokenizer.decode(input_2['input_ids'][1:-1])


In [33]:
text = 'În cele șapte etape ale Campionatului Național de Raliuri Dunlop, Simone a câștigat 47 de probe speciale, iar performanța de la Iași reeditează pe ceea precedentă, din Raliul Aradului.'
correct(text)

'În cele șapte etape ale Campionatului Național de Raliuri Dunlop, Simone a câștigat 47 de probe speciale, iar performanța de la Iași reeditează pe cea precedentă, din Raliul Aradului.'

In [9]:
'''file_new = open('/content/drive/MyDrive/Colab Notebooks/corpus/results.txt', "w")
file_new.write('')
file_new = open('/content/drive/MyDrive/Colab Notebooks/corpus/results.txt', "a")'''

'file_new = open(\'/content/drive/MyDrive/Colab Notebooks/corpus/results.txt\', "w")\nfile_new.write(\'\')\nfile_new = open(\'/content/drive/MyDrive/Colab Notebooks/corpus/results.txt\', "a")'

In [10]:
'''with open('/content/drive/MyDrive/Colab Notebooks/corpus/test.txt') as f:
    i = 0
    for line in f:
      #print(line)
      if i % 2 == 0:
        #print('C = ' + str(line))
        file_new.write('C = ' + str(line))
      else:
        #print('W = ' + str(line))
        #print('N = ' + correct(line) + '\n')
        file_new.write('W = ' + str(line))
        file_new.write('N = ' + correct(line) + '\n')
      i += 1'''

"with open('/content/drive/MyDrive/Colab Notebooks/corpus/test.txt') as f:\n    i = 0\n    for line in f:\n      #print(line)\n      if i % 2 == 0:\n        #print('C = ' + str(line))\n        file_new.write('C = ' + str(line))\n      else:\n        #print('W = ' + str(line))\n        #print('N = ' + correct(line) + '\n')\n        file_new.write('W = ' + str(line))\n        file_new.write('N = ' + correct(line) + '\n')\n      i += 1"